In [1]:
from utils_ot import *
from PIL import Image
import torch
from torchvision.transforms.functional import resize, to_tensor, to_pil_image
from models import *


In [2]:
ref_image_path = "images/zebra.png"
path_fake = "outputs/zebre/gen_20250610_160458_7.png"


In [3]:

im = Image.open(ref_image_path).convert("RGB")
im_tensor_cpu = to_tensor(im).to(device="cpu")
print(f"inital tensor size : {im_tensor_cpu.shape}")

max_dim = 25
_, H, W = im_tensor_cpu.shape
scale_factor = max_dim / max(H, W)

new_H = int(H * scale_factor)
new_W = int(W * scale_factor)
im_resized = resize(im_tensor_cpu.unsqueeze(0), size=[new_H, new_W], antialias=True)*2 -1
print(f"redized iamge shape : {im_resized.shape}")

inital tensor size : torch.Size([3, 380, 572])
redized iamge shape : torch.Size([1, 3, 16, 25])


In [4]:
im_fake = Image.open(path_fake).convert("RGB")
im_fake_tensor = to_tensor(im_fake).to(device="cpu")
print(f"inital tensor size : {im_fake_tensor.shape}")
print(im_resized.shape[2:])
im_fake_tensor = resize(im_fake_tensor.unsqueeze(0),size = im_resized.shape[2:],antialias=True)*2 - 1


inital tensor size : torch.Size([3, 166, 250])
torch.Size([16, 25])


# PIXEL PATCH SWD  

In [5]:
random_image = torch.rand_like(im_resized)


In [6]:
original_patches = extract_patches_batched(im_resized,10,10)

random_patches = extract_patches_batched(random_image * 2 -1  ,10,10)

fake_patches = extract_patches_batched(im_fake_tensor,10,10)



In [7]:
swd_f_random = sliced_wasserstein_distance(original_patches,random_patches,1000)
swd_fake = sliced_wasserstein_distance(original_patches,fake_patches,1000)

In [8]:
swd_f_random

tensor(0.4951)

In [9]:
swd_fake

tensor(0.0296)

# FEATURE PATCH SWD

In [10]:
inception_model = InceptionV3_Multi()
inception_model.eval()

import torch.nn.functional as F

def preprocess_for_inception(x):
    # accepte x en [3,H,W] ou [N,3,H,W]
    was_3d = (x.dim() == 3)
    if was_3d:
        x = x.unsqueeze(0)          # → [1,3,H,W]
    x = (x + 1) / 2
    x = F.interpolate(x, size=(299,299), mode='bilinear', align_corners=False)
    mean = torch.tensor([0.485,0.456,0.406], device=x.device).view(1,3,1,1)
    std  = torch.tensor([0.229,0.224,0.225], device=x.device).view(1,3,1,1)
    x = (x - mean) / std
    return x.squeeze(0) if was_3d else x

def preprocess_features(features):
    """
    Prend features de forme (B, C, H, W)
    et renvoie un tenseur (B * H * W, C),
    i.e. un nuage de B*H*W vecteurs C-dimensionnels.
    """
    B, C, H, W = features.shape
    # 1) permute pour déplacer C en dernière dimension
    x = features.permute(0, 2, 3, 1)    # [B, H, W, C]
    # 2) flatten batch+spatial en une seule dimension
    return x.reshape(B * H * W, C)      # [B*H*W, C]


In [13]:
fake_preprocessed = preprocess_for_inception(im_fake_tensor)      # shape [b,3,299,299], requires_grad=True
real_preprocessed = preprocess_for_inception(im_resized)  # idem, ou detach si tu préfères
random_preprocessed = preprocess_for_inception(random_image*2 - 1)
    
fake_feats = inception_model(fake_preprocessed)    
real_feats = inception_model(real_preprocessed)
random_feats = inception_model(random_preprocessed)
# ------------------------------------------------------------------
#  Boucle sur les différentes tailles de patch
# ------------------------------------------------------------------
for i in range(len(fake_feats)):
    swd_features_real = sliced_wasserstein_distance(preprocess_features(real_feats[i]), preprocess_features(fake_feats[i]), num_projections= 500)
    swd_features_random = sliced_wasserstein_distance(preprocess_features(real_feats[i]), preprocess_features(random_feats[i]), num_projections= 500)
    print(f"\nswd real : {swd_features_real} ")
    print(f"\nswd random : {swd_features_random} ")




swd real : 0.017706921324133873 

swd random : 0.1869531124830246 

swd real : 0.016321450471878052 

swd random : 0.07398839294910431 

swd real : 0.14348363876342773 

swd random : 0.5297847390174866 


In [12]:
swd_features_random

tensor(0.5703)

In [158]:
swd_features_real

tensor(0.1462, grad_fn=<MeanBackward0>)